## RGB Cutouts Demo

This notebook is intended to demonstrate how to make true-color cutouts from HSC data. It uses Robert Lupton's [Colour Images notebook](https://github.com/RobertLuptonTheGood/notebooks/blob/2eeee8b9fe35077387485e488c965f1ea3d39418/Demos/Colour%20Images.ipynb) as a base.

It illustrates how to:
* Access datasets using Butler
* Display images using lsst.afw.display
* Produce a RGB color cutout from the coadd


### Setup

This tutorial is meant to be run from the jupyterhub interface where the LSST stack is pre-installed.

We begin by importing packages from the LSST stack.

In [ ]:
# LSST Stack imports 

import lsst.daf.persistence as dafPersist
import lsst.afw.display.rgb as afwRgb
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage

Next, we import matplotlib and set up parameters for the figure sizes.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13, 8)

### Creating a Butler client

Butler is used to access processed image files in the LSST Pipelines. The Butler client is imported from lsst.daf.persistence and is set up below.

The coadd from the butler client is also defined below as the skymap.

In [ ]:
dataPath = "/datasets/hsc/repo/rerun/DM-13666/WIDE/"

butler = dafPersist.Butler(dataPath)
skymap = butler.get("deepCoadd_skyMap")

In [ ]:
# for Butler documentation
# help(butler)

Create a list of dataset types, including calexp and coadds.

In [ ]:
datasettypes = ['calexp', 'calexpBackground', 'icSrc',
               'src', 'srcMatch', 'srcMatchFull', 'ossImage',
               'flattenedImage', 'wcs', 'fcr', 'photoCalib',
               'jointcal_wcs', 'jointcal_photoCalib', 'skyCorr',
               'calexp_camera', 'brightObjectMask', 'deepCoadd_calexp',
               'deepCoadd_det', 'deepCoadd_meas', 'deepCoadd_measMatch',
               'deepCoadd_mergeDet', 'deepCoadd_ref', 'deepCoadd_forced_src',
               'forced_src' ]

Select the object of which we want a RGB image cutout.

For coadds the WCS is the same in all bands, but the code handles the general case

In [ ]:
ra, dec, name = 215.9747, -0.4344, "Lens"

raDec = afwCoord.Coord(ra*afwGeom.degrees, dec*afwGeom.degrees)

filters = "grizy"  # filters to process -- we choose our bands when we set B, R, G = ...

cutoutSize = 500   # pixels

Start by finding the tract and patch

In [ ]:
for i, tp in enumerate(skymap.findTractPatchList([raDec])):
    tractInfo, patchInfo = tp
    tract = tractInfo.getId()
    patch = "%d,%d" % patchInfo[0].getIndex()
    print i, tract, patch

Then we can read the desired pixels

In [ ]:
images = {}
cutoutSize = afwGeom.ExtentI(300, 200)

for f in filters:
    filterName = "HSC-%s" % f.upper()
    md = butler.get("deepCoadd_calexp_md", immediate=True,
                    tract=tract, patch=patch, filter=filterName)
    wcs = afwImage.makeWcs(md)
    xy = afwGeom.PointI(wcs.skyToPixel(raDec))

    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

    images[f] = butler.get("deepCoadd_calexp_sub", bbox=bbox, immediate=True,
                            tract=tract, patch=patch, filter=filterName).getMaskedImage()

Generate a RGB images, and optionally write to disk

In [ ]:
rgbFileFmt = "%s-%%s.png" % name if False else None
if not False:
    min = dict(gri=0.01, riz=0.01, izy=0.01)
    max = dict(gri=0.20, riz=0.20, izy=0.25)
else:
    min = dict(gri=0.01, riz=0.01, izy=0.05)
    max = dict(gri=0.20, riz=0.40, izy=0.50)

Q = 10

for bands in ["gri", "riz", "izy"]:
    B, G, R = bands
    rgb = afwRgb.makeRGB(images[R], images[G], images[B],
                         min[bands], max[bands] - min[bands], Q,
                         #saturatedBorderWidth=1, saturatedPixelValue=10
                         )
    
    afwRgb.displayRGB(rgb)
    
    if rgbFileFmt:
        afwRgb.writeRGB(rgbFileFmt % bands, rgb)